In [ ]:
import sys
nb_path = '/content/drive/MyDrive/gpt2'
sys.path.insert(0, nb_path) 

In [ ]:
%cd '/content/drive/MyDrive/gpt2'

/content/drive/MyDrive/gpt2


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.5 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 32.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [ ]:
!nvidia-smi

Tue Apr 26 14:14:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.manual_seed(42)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium',bos_token="<|BOS|>",
                    eos_token="<|EOS|>",
                    unk_token="<|UNK|>",                    
                    pad_token="<|PAD|>",
                    sep_token="<|SEP|>")
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Embedding(50261, 1024)

In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/Commentary_final_dataset_final.csv')['Text']


In [ ]:
dataset=dataset[:20000]
dataset

0        <|BOS|>australia matt renshaw pakistan mohamma...
1        <|BOS|>australia david warner pakistan mohamma...
2        <|BOS|>australia david warner pakistan mohamma...
3        <|BOS|>australia david warner pakistan mohamma...
4        <|BOS|>australia matt renshaw pakistan mohamma...
                               ...                        
19995    <|BOS|>australia hilton cartwright pakistan im...
19996    <|BOS|>australia hilton cartwright pakistan im...
19997    <|BOS|>australia peter handscomb pakistan yasi...
19998    <|BOS|>australia peter handscomb pakistan yasi...
19999    <|BOS|>australia peter handscomb pakistan yasi...
Name: Text, Length: 20000, dtype: object

In [ ]:
max_length = max([len(tokenizer.encode(str(d))) for d in dataset])

In [ ]:
# Dataset preparation for GPT-2 model
class commentary_dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(txt,max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
### Spliting dataset into training and validation
data = commentary_dataset(dataset, tokenizer, max_length=max_length)
train_size = int(0.9 * len(data))
train_dataset, val_dataset = random_split(data, [train_size, len(data) - train_size])

In [ ]:
len(data)

2156

In [ ]:
import gc
gc.collect()

286

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#training arguments for trainer class
training_args = TrainingArguments(
    output_dir="/content/",num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [ ]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

/content/drive/MyDrive/gpt2/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1940
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1940


Step,Training Loss
100,2.758600
200,0.743300
300,0.489500
400,0.433900
500,0.364300
600,0.345300
700,0.352900
800,0.325000
900,0.307400
1000,0.279900




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1940, training_loss=0.471460127584713, metrics={'train_runtime': 1697.244, 'train_samples_per_second': 1.143, 'train_steps_per_second': 1.143, 'total_flos': 703780995072000.0, 'train_loss': 0.471460127584713, 'epoch': 1.0})

In [ ]:
trainer=Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 609
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 609


Step,Training Loss
100,0.362500
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=609, training_loss=0.059521780992477805, metrics={'train_runtime': 242.0946, 'train_samples_per_second': 2.516, 'train_steps_per_second': 2.516, 'total_flos': 331393782988800.0, 'train_loss': 0.059521780992477805, 'epoch': 1.0})

In [ ]:
prompt="<|BOS|>india sachin pakistan 50 run four 6degree 88 70<|SEP|>"
generated = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=50, top_p=0.95, temperature=1.9, num_return_sequences=20
                                )

In [ ]:
for i, sample_output in enumerate(sample_outputs):
  text=tokenizer.decode(sample_output, skip_special_tokens=True)
  x=len(prompt)
  print("{}: {}".format(i+1,text[x-14:]))

1: runs from underneath. played beautifully and flat
2: swept down towards the batsman who nudged after a drive
3: shrinking more than four feet, waram is out at his hips in defense too well as the fielder leaves his ball into four
4: driven through gully from deep to square drive to cover for warner
5: another flat but very straight, worked up his loop and smited to d'oh, worked through squaredrive
6: another round stroke. played. a low low length. more and louder ball that should leave bat man but is dragged away behind the cut to middle stump
7: 90 pll off to nipped through square drive point
8: bases one down to dab between square leg, drives one towards pkwickets. with it still intact
9: another full delivery down leg, the width doesn there. blocked another low ball. not quite straight. well struck
10: another square paddleslowly beats his warbling drive at the right end but younis can hardly believe the appeal. warly drive past gushing tea cart
11: tight delivery outside off with

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/nlp_project/checkpoint-5000",num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = '/content/drive/MyDrive/automated_comment_system'
# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

Configuration saved in /content/drive/MyDrive/automated_comment_system/config.json
Model weights saved in /content/drive/MyDrive/automated_comment_system/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/automated_comment_system/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/automated_comment_system/special_tokens_map.json
added tokens file saved in /content/drive/MyDrive/automated_comment_system/added_tokens.json


('/content/drive/MyDrive/automated_comment_system/tokenizer_config.json',
 '/content/drive/MyDrive/automated_comment_system/special_tokens_map.json',
 '/content/drive/MyDrive/automated_comment_system/vocab.json',
 '/content/drive/MyDrive/automated_comment_system/merges.txt',
 '/content/drive/MyDrive/automated_comment_system/added_tokens.json')

In [ ]:
# loading the model and tokenizer

output_dir = '/content/drive/MyDrive/automated_comment_system'

device = torch.device('cuda:0')

tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained(output_dir)

model.to(device)

prompt="|<BOS>| australia david warner pakistan mohammad amir 4 4 3rd man 50 145 140 |<SEP|>"
generated = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=70, top_p=0.95, temperature=1.9, num_return_sequences=20)

for i, sample_output in enumerate(sample_outputs):
  text=tokenizer.decode(sample_output, skip_special_tokens=True)
  x=len(prompt)
  print("{}: {}".format(i+1,text[x:]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1: swept and clipped around backward boundary off the pads. blocked
2: another bounce on middle which warner doesn't mind and just drives straight into a block boundary swing
3: worrying warner just pulled off from third man but good swing in. left alone <b>[145 runs up to start this thing. nice hands. defended towards the end.
4: on a widthy stride at mid-point with arms folded from warner, slashed through pender for a couple
5: warner comes over and misses a pull drive on a wide
6: full, no chance against mid-off from the crease, defended into a second fine length boundary wakhar wak no man of the line
7:  warner doesn't quite swing to the third man and gets out at third man point. more time in front of point on the outside, a good defence
8: 140 kph. nice and well around off. nice line too but what you need is a partner of 130 of such ease because the field is good, and a very handsome touch from
9: played to three more at backward second man</strong> by mohammad. great line for sha